In [2]:
!pip install --upgrade gradio -q


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import gradio as gr

def shout(name):
    return name.upper()

demo = gr.Interface(
    fn=shout,
    inputs="textbox",
    outputs="textbox",
    flagging_mode="never",
)

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://65993d41da1661ac01.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
import gradio as gr

def shout(name):
    return name.upper()

demo = gr.Interface(
    fn=shout,
    inputs=gr.Textbox(label="Your response: ", lines=8),
    outputs=gr.Textbox(label="Response: ", lines=6),
    flagging_mode="never",
)

demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [18]:
import gradio as gr

def prompt(text, model):
    texts = [
        "johni",
        "johin douglas"
    ]


    for t in texts:
        yield t
       
demo = gr.Interface(
    fn=prompt,
    inputs=[gr.Textbox(label="Question"), gr.Dropdown( ["GPT", "Claude"], label="Select the model")],
    outputs=[gr.Markdown(label="Result")],
    flagging_mode="never",
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


## Chat Interface

In [2]:
import random

def random_response(message, history):
    print(history)
    return random.choice(["Yes", "No"])

import gradio as gr

gr.ChatInterface(
    fn=random_response, 
    type="messages"
).launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


[]
[{'role': 'user', 'metadata': None, 'content': 'Hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'No', 'options': None}]


## Chat with Tools

In [36]:
from openai import OpenAI
import json

model = "gpt-4o-mini"

client = OpenAI(api_key="sk-proj-")

In [37]:
system_message = """You are a helpful assistant for an Airline called FlightAI. 
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [38]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [39]:
ticket_price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [40]:
tools = [{"type": "function", "function": ticket_price_function}]

In [41]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model=model, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        print("Message: ", message)
        print("Response: ", response)
        response = client.chat.completions.create(model=model, messages=messages)
    
    return response.choices[0].message.content

In [42]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [43]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
Message:  ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Ih7v9E2Vla5HhXOLdUThxOrS', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')])
Response:  {'role': 'tool', 'content': '{"destination_city": "London", "price": "$799"}', 'tool_call_id': 'call_Ih7v9E2Vla5HhXOLdUThxOrS'}
